## Train_Validation_Split

In [5]:
# Import modules
import os
import random
import shutil


img_source_dir = '../../datasets/test'
img_target_dir = '../../datasets/data' 

train_size = 0.90


if not os.path.exists(img_target_dir):
    os.makedirs(img_target_dir)
else:
    if not os.path.exists(img_target_dir + '/train'):
        os.makedirs(img_target_dir + '/train')
    if not os.path.exists(img_target_dir + '/validation'):
        os.makedirs(img_target_dir + '/validation')

        
subdirs = [subdir for subdir in os.listdir(img_source_dir) if os.path.isdir(os.path.join(img_source_dir, subdir))]


for subdir in subdirs:
    subdir_fullpath = os.path.join(img_source_dir, subdir)
    train_subdir = os.path.join(img_target_dir + '/train', subdir)
    validation_subdir = os.path.join(img_target_dir + '/validation', subdir)
    
    if not os.path.exists(train_subdir):
        os.makedirs(train_subdir)
        
    if not os.path.exists(validation_subdir):
        os.makedirs(validation_subdir)
        
    train_counter = 0
    validation_counter = 0
    
    # Randomly assign an image to train or validation folder
    for filename in os.listdir(subdir_fullpath):
        if filename.endswith(".jpg") or filename.endswith(".png"): 
            fileparts = filename.split('.')
            
            if random.uniform(0, 1) <= train_size:
                shutil.move(os.path.join(subdir_fullpath, filename), os.path.join(train_subdir, str(train_counter) + '.' + fileparts[1]))
                train_counter += 1
            else:
                shutil.move(os.path.join(subdir_fullpath, filename), os.path.join(validation_subdir, str(validation_counter) + '.' + fileparts[1]))
                validation_counter += 1

---

## Seed image Crop

In [33]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
import os
import glob

label_path = './test/xml'
xml_list = os.listdir(label_path)

boxes = []
cnt = 0
for xml_file in sorted(xml_list):
    if xml_file =='.DS_Store':
        pass
    else:
            #try:
        xml_path = os.path.join(label_path,xml_file)

        root_1 = minidom.parse(xml_path)
        bnd_1 = root_1.getElementsByTagName('bndbox')

        for i in range(len(bnd_1)):
            xmin = int(bnd_1[i].childNodes[1].childNodes[0].nodeValue)
            ymin = int(bnd_1[i].childNodes[3].childNodes[0].nodeValue)
            xmax = int(bnd_1[i].childNodes[5].childNodes[0].nodeValue)
            ymax = int(bnd_1[i].childNodes[7].childNodes[0].nodeValue)
            
            boxes.append((xmin,ymin,xmax,ymax))
        

In [32]:
boxes_1

[(62, 277, 929, 2448),
 (1346, 508, 1773, 2371),
 (1917, 661, 2235, 2041),
 (2326, 800, 2580, 1929),
 (2616, 900, 2807, 1847),
 (2826, 914, 2948, 1743),
 (2962, 959, 3084, 1693),
 (3096, 1023, 3178, 1646)]

In [ ]:
# 이미지 리스트 처음에 정의해야 함 - 리스트 가져와서 자동으로 쓰는 형식..
#li = ['baeksansoo','samdasoo','e_liter','sprite','trevi','pocari_pet','ion_the_fit','evian']

#li = ['grape','peach','gal_bae','mango','vita_500','dailyC','hongsam','gas_hwal']   #1단
li = ['pocari1','pocari2','power1','power2','red_bull','2%','sol','tejava']   # 2단
#li = ['coca_can1','coca_can2','pepsi_can','milkis','demisoda','oranC','sprite','welchs'] # 3단
#li = ['coca_pet1','coca_pet2','pepsi_pet1','pepsi_pet2','sprite_pet1','sprite_pet2','gal_bae_pet1','gal_bae_pet2']
#li = ['2%_peach_pet1','2%_peach_pet2','fanta_orange_pet1','fanta_orange_pet2','fanta_pine_pet','mccol','milkis_pet1','milkis_pet2']
#li = ['aloe_pet1','aloe_pet2','apple_pet','virak_pet','mogumogu','lemonade','jeju_pet1','jeju_pet2']
#li = ['potato_original1','potato_original2','potato_original3','sweet_potato1','sweet_potato2','sweet_potato3','potato_onion1','potato_onion2']
#li = ['oyes','fresh_berry']
#li = ['potato_stick1','potato_stick2','potato_chips1','potato_chips2']

In [23]:
import os
from PIL import Image
from tqdm import tqdm


SEED_IMAGE_PATH = './test/seed_image/'        # seed data 있는 곳
CLS_IMAGE_PATH = './test/result/'             # 잘린 데이터 저장할 곳

item_total_list = [li[i:] + li[:i] for i in range(len(li))]
seed_image_list = os.listdir(SEED_IMAGE_PATH)


for folder in li:
    folder_path = CLS_IMAGE_PATH + folder
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

def get_index(product):
    return [i[0] == product for i in item_total_list].index(True)

def crop_train_data(boxes, resize=None):
    num = 0
    #seed_image_dir = os.listdir('./seed_image/')
    
    for i in tqdm(range(len(seed_image_list)), desc="processing"):
        #seed_image_dir = os.listdir('./train_images/crop_train/')
        #img_path = './test/seed_image/{}/'.format(seed_image_list[i])  # 백산수, travi..
        img_path = SEED_IMAGE_PATH + '{}/'.format(seed_image_list[i])
        img_list = os.listdir(img_path)

        #idx = get_index(image_dir[i].split('.')[1])
        idx = get_index(seed_image_list[i])
        item_list = item_total_list[idx]

        for img in img_list:
            image = os.path.join(img_path,img)
            input_img = Image.open(image)

            for k in range(4):
                result = input_img.crop(boxes[k])
                result = result.resize(resize)
                #result.save('./test/result/{}/{}.jpg'.format(item_list[k],num))
                result.save(CLS_IMAGE_PATH + '{}/{}.jpg'.format(item_list[k],num))
                num = num + 1
            

In [24]:
crop_train_data(boxes_1, resize=(224,224))

processing: 100%|██████████| 8/8 [02:20<00:00, 17.62s/it]


---

## Convert xml to txt

In [ ]:
# -*- coding: utf-8 -*-

from xml.dom import minidom
import os
import glob

In [ ]:
lut={}
lut["fresh berry"]      =0
lut["ace"]              =1
lut["sand"]             =2
lut["oreo"]             =3
lut["choco"]            =4
lut["homerun_milk"]     =5
lut["potato_onion"]     =6
lut["chocochip"]        =7
lut["potato stick"]     =8
lut["potato chips"]     =9
lut["homerun_choco"]    =10
lut["coconut"]          =11
lut["oyes"]             =12
lut["potato_original"]   =13
lut["zec"]              =14
lut["homerun_tiramisu"] =15
lut["sweet potato"]     =16

In [ ]:
def convert_coordinates(size, box):
    dw = 1.0/size[0]
    dh = 1.0/size[1]
    x = (box[0]+box[1])/2.0
    y = (box[2]+box[3])/2.0
    w = box[1]-box[0]
    h = box[3]-box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)


def convert_xml2yolo( lut ):

    for fname in glob.glob("./labels/*.xml"):
        
        xmldoc = minidom.parse(fname)
        
        fname_out = (fname[:-4]+'.txt')

        with open(fname_out, "w") as f:

            itemlist = xmldoc.getElementsByTagName('object')
            size = xmldoc.getElementsByTagName('size')[0]
            width = int((size.getElementsByTagName('width')[0]).firstChild.data)
            height = int((size.getElementsByTagName('height')[0]).firstChild.data)

            for item in itemlist:
                # get class label
                classid =  (item.getElementsByTagName('name')[0]).firstChild.data
                if classid in lut:
                    label_str = str(lut[classid])
                else:
                    label_str = "-1"
                    print ("warning: label '%s' not in look-up table" %classid)

                # get bbox coordinates
                xmin = ((item.getElementsByTagName('bndbox')[0]).getElementsByTagName('xmin')[0]).firstChild.data
                ymin = ((item.getElementsByTagName('bndbox')[0]).getElementsByTagName('ymin')[0]).firstChild.data
                xmax = ((item.getElementsByTagName('bndbox')[0]).getElementsByTagName('xmax')[0]).firstChild.data
                ymax = ((item.getElementsByTagName('bndbox')[0]).getElementsByTagName('ymax')[0]).firstChild.data
                b = (float(xmin), float(xmax), float(ymin), float(ymax))
                bb = convert_coordinates((width,height), b)
                #print(bb)

                f.write(label_str + " " + " ".join([("%.6f" % a) for a in bb]) + '\n')

        print ("wrote %s" % fname_out)



def main():
    convert_xml2yolo( lut )


if __name__ == '__main__':
    main()